In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [2]:
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [3]:
A_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Technology%20Matrix%20(A)%20-%20Design.csv"
B_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Environmental%20Matrix%20(B)%20-%20Design.csv"
C_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Characterization%20Matrix%20(C)%20-%20Design.csv"

In [4]:
A_df = pd.read_csv(A_url)
B_df = pd.read_csv(B_url)
C_df = pd.read_csv(C_url)

In [5]:
A_df_org = A_df
B_df_org = B_df
C_df_org = C_df

In [6]:
#Building A matrix
# Step 1: Delete the first 4 columns
A_df = A_df.drop(A_df.columns[:4], axis=1)

# Step 2: Delete the first 4 rows (first row is heading, so put 3)
A_df = A_df.iloc[3:]

#Replacing empty values with zero and getting the final A matrix
A=A_df
A=A.replace(np.nan, 0)
A=np.array(A,dtype='float64')

In [7]:
#Building B matrix
# Step 1: Delete the first 6 columns
B_df = B_df.drop(B_df.columns[:6], axis=1)

# Step 2: Delete the first 4 rows (first row is heading, so put 3)
B_df = B_df.iloc[3:]

#Replacing empty values with zero and getting the final B matrix
B=B_df
B=B.replace(np.nan, 0)
B=np.array(B,dtype='float64')

In [8]:
#Building C matrix
# Step 1: Delete the first 6 columns
C_df = C_df.drop(C_df.columns[:6], axis=1)

# Step 2: Delete the first 5 rows (first row is heading, so put 3)
C_df = C_df.iloc[4:]

#Replacing empty values with zero and getting the final B matrix
C=C_df
C=C.replace(np.nan, 0)
C=np.array(C,dtype='float64')

In [48]:
#Functional Unit
# F = 168.450 million metric tons = 168,450,000 metric tons = 168,450,000,000 kg
F=168450000000
F=int(F)
#defining f matrix
f=np.zeros(len(A))
f[0]=F

In [61]:
C_gwp= np.transpose(C)[[0]]
coef_GWP=C_gwp@B
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

In [62]:
#Model Formulation
# Create the model
model = ConcreteModel()
# List of processes with negative scaling factor due to open loop recovery and substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements = [
    'textile production, nonwoven polyester, needle-punched | textile, nonwoven polyester | APOS, S',
    'market for sawlog and veneer log, softwood, debarked, measured as solid wood | sawlog and veneer log, softwood, debarked, measured as solid wood | APOS, S',
    'pitch production, petroleum refinery operation | pitch | APOS, S',
    'lignite mine operation | lignite | APOS, S',
    'market group for electricity, medium voltage | electricity, medium voltage | APOS, S',
    'heat production, natural gas, at industrial furnace low-NOx >100kW | heat, district or industrial, natural gas | APOS, S',
    'naphtha production, petroleum refinery operation | naphtha | APOS, S',
    'methanol production | methanol | APOS, S',
    'petroleum production, onshore | petroleum | APOS, S'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_org.iloc[:,0].tolist().index(elem) for elem in search_elements]
negative_s_indices = [i-3 for i in negative_s_indices]
all_s_indices = list(range(1, len(np.transpose(A))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_s_indices]

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A)))
model.s = Var(model.set_s)
model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)

#model constraint: As = f and s>=0
model.set_balance = RangeSet(len(f))
def balance(model, p): # As = f
    return sum(A[p-1,i-1]*model.s[i] for i in model.set_s) == f[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)

model.balance_constraints = Constraint(model.set_balance, rule=balance)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)

model.obj = Objective(expr = sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=minimize)
#Solver
solver = SolverFactory('glpk')
solver.solve(model) # solves and updates instance

{'Problem': [{'Name': 'unknown', 'Lower bound': 655493249414.225, 'Upper bound': 655493249414.225, 'Number of objectives': 1, 'Number of constraints': 261, 'Number of variables': 144, 'Number of nonzeros': 612, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.08178901672363281}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [63]:
value(model.obj)

655493249414.2256

In [64]:
scaling_factors = []
for j in model.s:
    scaling_factors.append(model.s[j].value)  

In [66]:
# Sankey Diagram
#data
label = ["Resources","Feedstocks","Fossil-based monomers","Bio-based monomers","Monomers",
        "Polymers","Products","Use & collection","Sorting","Advanced Recycling", "Mechanical Recycling",
        "Chemical Recycling", "Downcycling", "Energy Recovery", "Landfill", "Mismanaged Waste",
        "Macroplastics", "Microplastics"]

#1: Resources
#2: Feedstocks
#3: Fossil-based monomers
#4: Bio-based monomers
#5: Monomers
#6: Polymers
#7: Products
#8: Use & collection
#9: Sorting
#10: Advanced Recycling
#11: Mechanical Recycling
#12: Chemical Recycling
#13: Downcycling
#14: Energy Recovery
#15: Landfill
#16: Mismanaged Waste
#17: Macroplastics
#18: Microplastics

source = [1,2,2,3,4,5,6,7,8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9,10,10,10,10,11,11,11,11,12,12,12,12,12,13,13,13,16,17]
target = [2,3,4,5,5,6,7,8,9,14,15,16,17,10,11,12,13,14,15,16, 7,14,15,16, 7,14,15,16, 5, 2,14,15,16,14,15,16,17,18]
